In [1]:
from korean_crawling_bundles import *

In [2]:
train_df = pd.read_csv('0921_with_link.csv')

In [3]:
word_df = pd.read_csv('konlpy_0921_link.csv')
word_df.head()

,0
0,"['최종', '최종구', '구', '위원장', '내년', '4', '4월', '월'..."
1,"['김', '김동연', '동연', '고용', '상황', '바닥', '중', '하반기..."
2,"['정부', '입', '입국장', '국장', '면세점', '도입', '방안', '추..."
3,"['기재', '기재차관', '차관', '고용', '개선', '가용', '정책', '..."
4,"['집값', '대책', '우리', '해외', '해외서도', '서도', '금융', '..."


In [4]:
for i in range(len(word_df)) : 
    word_df.iat[i,0] = word_df.iat[i,0][1:-1]
    
word_ls = []
for i in range(len(word_df)) : 
    new_ls = word_df.iat[i,0].split(',')

    for i in range(len(new_ls)) : 
        new_ls[i] = new_ls[i].replace(' ','')
    
    for i in range(len(new_ls)) : 
        new_ls[i] = new_ls[i].replace("'",'')
    
    word_ls += new_ls
print(len(word_ls))

for i in range(len(word_df)):
    word_df.iat[i,0] = ''.join(word_df.iat[i,0]).replace("'","").split(',')
    
for i in range(len(word_df)):
    word_df.iat[i,0] = ','.join(word_df.iat[i,0]).replace(' ','').split(',')

19335


In [5]:
word_df.rename(columns={'0':'corpus'},inplace=True)
word_df.tail()

,corpus
82,"[머니, 머니투데이, 투데이, 김, 김소연, 소연, 기자, 개, 개장전, 장전, 뉴..."
83,"[아시아, 아시아경제, 경제, 조호, 조호윤, 윤, 기자, 자본, 자본시장, 시장,..."
84,"[머니, 머니투데이, 투데이, 뉴욕, 미국, 송, 송정렬, 정렬, 특파원, 다, 존..."
85,"[머니, 머니투데이, 투데이, 뉴욕, 미국, 송, 송정렬, 정렬, 특파원, 뉴욕증시..."
86,"[이, 이데일리, 데일리, 뉴스, 뉴스속보팀, 속보, 팀, 유럽, 주요, 증시, 2..."


In [6]:
print(len(word_df.iat[0,0]))
for _ in range(10):
    for i in range(len(word_df)):
        for idx,j in enumerate(word_df.iat[i,0]):
            if len(j) < 2 : 
                word_df.iat[i,0].pop(idx)
print(len(word_df.iat[0,0]))

104
88


In [7]:
word_ls = []
for i in range(len(word_df)):
    word_ls += word_df.iat[i,0]
len(word_ls)

16247

In [8]:
from collections import Counter
cnt_dict = dict(Counter(word_ls))

df = pd.DataFrame(columns=['Count'])
df['Count'] = cnt_dict.values()
df.index = cnt_dict.keys()
df.sort_values('Count',ascending=False,inplace=True)

df.head()

,Count
기자,69
21,51
뉴스,48
미국,47
경제,47


In [9]:
import string
pt = string.printable
print(pt[:])

pt_ls = []
all_ls = []
for i in range(len(df)):
    all_ls = []
    for comp in df.index[i]:
        all_ls.append(comp)
        if all(x not in pt for x in all_ls) : 
            pt_ls.append(i)
pt_ls = list(set(pt_ls))

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	



In [10]:
df = df.iloc[pt_ls]

In [11]:
df

,Count
기자,69
뉴스,48
미국,47
경제,47
달러,44
관련,41
서울,41
이후,39
중국,39
무역,36


# LDA process

In [12]:
documents = [i for i in word_df['corpus']]
K = 5

In [13]:
from collections import Counter

# 각 토픽이 각 문서에 할당되는 횟수
# Counter로 구성된 리스트
# 각 Counter는 각 문서를 의미
document_topic_counts = [Counter() for _ in documents]

# 각 단어가 각 토픽에 할당되는 횟수
# Counter로 구성된 리스트
# 각 Counter는 각 토픽을 의미
topic_word_counts = [Counter() for _ in range(K)]

# 각 토픽에 할당되는 총 단어수
# 숫자로 구성된 리스트
# 각각의 숫자는 각 토픽을 의미함
topic_counts = [0 for _ in range(K)]

# 각 문서에 포함되는 총 단어수
# 숫자로 구성된 리스트
# 각각의 숫자는 각 문서를 의미함
document_lengths = [len(doc) for doc in documents]

# 단어 종류의 수
distinct_words = set(word for document in documents for word in document)
V = len(distinct_words)

# 총 문서의 수
D = len(documents)

In [14]:
def p_topic_given_document(topic, d, alpha=0.1):
    # 문서 d의 모든 단어 가운데 topic에 속하는
    # 단어의 비율 (alpha를 더해 smoothing)
    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):
    # topic에 속한 단어 가운데 word의 비율
    # (beta를 더해 smoothing)
    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + V * beta))

def topic_weight(d, word, k):
    # 문서와 문서의 단어가 주어지면
    # k번째 토픽의 weight를 반환
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

In [15]:
def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

import random
def sample_from(weights):
    # i를 weights[i] / sum(weights)
    # 확률로 반환
    total = sum(weights)
    # 0과 total 사이를 균일하게 선택
    rnd = total * random.random()
    # 아래 식을 만족하는 가장 작은 i를 반환
    # weights[0] + ... + weights[i] >= rnd
    for i, w in enumerate(weights):
        rnd -= w
        if rnd <= 0:
            return i

In [16]:
%%time
random.seed(0)

# topic 수 지정

# 각 단어를 임의의 토픽에 랜덤 배정
document_topics = [[random.randrange(K) for word in document]
                    for document in documents]

# 위와 같이 랜덤 초기화한 상태에서 
# AB를 구하는 데 필요한 숫자를 세어봄
for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

CPU times: user 46.9 ms, sys: 0 ns, total: 46.9 ms
Wall time: 46.7 ms


In [17]:
%%time
for iter in range(1000):
    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d],
                                              document_topics[d])):
            # 깁스 샘플링 수행을 위해
            # 샘플링 대상 word와 topic을 제외하고 세어봄
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1

            # 깁스 샘플링 대상 word와 topic을 제외한 
            # 말뭉치 모든 word의 topic 정보를 토대로
            # 샘플링 대상 word의 새로운 topic을 선택
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic

            # 샘플링 대상 word의 새로운 topic을 반영해 
            # 말뭉치 정보 업데이트
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1

CPU times: user 2min 51s, sys: 209 ms, total: 2min 52s
Wall time: 2min 52s


In [18]:
lda_df = pd.DataFrame()
k_ls = []
word_ls = []
count_ls = []
for k , word_counts in enumerate(topic_word_counts):
    for word,count in word_counts.most_common():
        k_ls.append(k)
        word_ls.append(word)
        count_ls.append(count)
lda_df['topic'] = k_ls
lda_df['word']  = word_ls
lda_df['count'] = count_ls

In [19]:
display(lda_df[lda_df['topic']==0].sort_values('count',ascending=False).head())
display(lda_df[lda_df['topic']==1].sort_values('count',ascending=False).head())
display(lda_df[lda_df['topic']==2].sort_values('count',ascending=False).head())
display(lda_df[lda_df['topic']==3].sort_values('count',ascending=False).head())
display(lda_df[lda_df['topic']==4].sort_values('count',ascending=False).head())

,topic,word,count
0,0,달러,30
2,0,제품,23
1,0,중국,23
3,0,관세,20
4,0,미국,19


,topic,word,count
4727,1,20,19
4728,1,기자,17
4729,1,이후,15
4730,1,경제,14
4731,1,미국,14


,topic,word,count
9511,2,상승,21
9512,2,무단전재,18
9513,2,지수,17
9514,2,거래,17
9515,2,무단,16


,topic,word,count
14223,3,기자,16
14224,3,21,15
14225,3,연합,14
14226,3,뉴스,14
14227,3,21일,13


,topic,word,count
19021,4,기자,19
19022,4,21,18
19023,4,2018.09,14
19024,4,서울,12
19025,4,사업,12


In [20]:
from collections import OrderedDict

In [21]:
for i in range(len(word_df)):
    document_topic_counts[i] = OrderedDict(document_topic_counts[i])
    
topic_ls = [idx for i in range(len(word_df)) for idx,k in enumerate(document_topic_counts[i].values()) \
 if k == np.max(list(((document_topic_counts[i]).values())))]

topic_count_ls = [k for i in range(len(word_df)) for idx,k in enumerate(document_topic_counts[i].values()) \
 if k == np.max(list(((document_topic_counts[i]).values())))]

In [22]:
word_df['topic'] = topic_ls
word_df['topic_count'] = topic_count_ls

In [23]:
word_df

,corpus,topic,topic_count
0,"[최종, 최종구, 위원장, 내년, 4월, 5월, 인터넷, 인터넷은행, 은행, 예비,...",4,79
1,"[김동연, 동연, 고용, 상황, 바닥, 하반기, 추세, 지속, 이데일리, 데일리, ...",4,98
2,"[정부, 입국장, 국장, 면세점, 도입, 방안, 추석, 추석연후, 발표, 연합, 연...",1,91
3,"[기재, 기재차관, 차관, 고용, 개선, 가용, 정책, 정책수단, 수단, 총동원, ...",4,220
4,"[집값, 대책, 우리, 해외, 해외서도, 서도, 금융, 금융규제, 규제, 강화, 헤...",0,237
5,"[박용, 회장, 평양, 모습, 인상적, 뉴시스, 시스, 김지은, 지은, 2018.0...",3,89
6,"[한진, 한진그룹, 그룹, 월급, 횡령, 이용, 주장, 사실, 뉴시스, 시스, 김동...",4,131
7,"[보따리, 보따리상, 서울, 시내, 면세점, 매출, 앞자리, 자리, 아시아, 아시아...",3,203
8,"[부동산, 규제, 강화, 건축, 비대, 비대위, 머니, 머니투데이, 투데이, 기자,...",1,261
9,"[보따리, 보따리상, 서울, 시내, 면세점, 매출, 앞자리, 자리, 아시아, 아시아...",0,202


In [24]:
topic_0_index = word_df[word_df['topic'] == 0].sort_values('topic_count',ascending=False).head(1).index.values[0]
topic_1_index = word_df[word_df['topic'] == 1].sort_values('topic_count',ascending=False).head(1).index.values[0]
topic_2_index = word_df[word_df['topic'] == 2].sort_values('topic_count',ascending=False).head(1).index.values[0]
topic_3_index = word_df[word_df['topic'] == 3].sort_values('topic_count',ascending=False).head(1).index.values[0]
topic_4_index = word_df[word_df['topic'] == 4].sort_values('topic_count',ascending=False).head(1).index.values[0]

In [25]:
topic_index_ls = []
topic_index_ls.append(topic_0_index)
topic_index_ls.append(topic_1_index)
topic_index_ls.append(topic_2_index)
topic_index_ls.append(topic_3_index)
topic_index_ls.append(topic_4_index)
print(topic_index_ls)
train_df.iloc[topic_index_ls]['Link']

[83, 85, 31, 40, 10]


83    https://finance.naver.com/news/news_read.nhn?a...
85    https://finance.naver.com/news/news_read.nhn?a...
31    http://finance.daum.net/news/news_content.daum...
40    https://finance.naver.com/news/news_read.nhn?a...
10    http://finance.daum.net/news/news_content.daum...
Name: Link, dtype: object

In [26]:
train_df.iloc[topic_index_ls]['Link'].values

array([ 'https://finance.naver.com/news/news_read.nhn?article_id=0004320921&office_id=277&mode=mainnews&type=&date=2018-09-21&page=1',
       'https://finance.naver.com/news/news_read.nhn?article_id=0004109156&office_id=008&mode=mainnews&type=&date=2018-09-21&page=1',
       'http://finance.daum.net/news/news_content.daum?type=main&sub_type=&docid=MD20180921054536340&section=&limit=30',
       'https://finance.naver.com/news/news_read.nhn?article_id=0004109156&office_id=008&mode=mainnews&type=&date=2018-09-21&page=1',
       'http://finance.daum.net/news/news_content.daum?type=main&sub_type=&docid=MD20180921092527777&limit=30&page=1'], dtype=object)

In [29]:
train_df.iloc[topic_index_ls]['Title'].values

array([nan, nan, '[뉴욕마감]뉴욕증시, 무역우려 완화·기술주 강세에 상승..다우·S&P, 사상 최고가', nan,
       "부동산 규제 강화에 '재건축 비대위' 힘받나"], dtype=object)